# Quadratic Linearly Constrained Binary Optimization
## Introduction
Quadratic Unconstrained Binary Optimization (Qubo) problems can be formulated from a square, symmetric objective function and a matrix of binary constraints. Suppose we are given an objective function, $O$, of dimension $n \times n$,  and a set of $m$ constraints, represented by the matrix $A$, with dimension $m \times n$ and right-hand side vector $b$ of length $m$. We want to combine them into a Qubo, which can be defined as $Q = O + \alpha (A^T A-2\mathrm{diag}(b^TA))$, where $\alpha \in \mathbb{R}$. At this point, we can find an optimal solution,
$x^{*} = \min_{x} x^T Q x$. 
The parameter $\alpha$ plays an important role in guaranteeing that the constraints are satisfied. We will not go into more detail on this page. 
We will define a simple problem on the Upload tab and show how to upload the components. The examples on the Running tab will describe how to run the constraint problem. 
Suppose the original problem we want to minimize is 
$-3xy + xz,$
subject to the constraints $x + z = 1$ and  $2x + 2y= 2$.




## Uploading
There are three matrix components that can be associated with to a constraint problem of this type. The objective function in matrix form, the linear constraints matrix, and an optional right-hand side (RHS) represent the linear constraints themselves. This can optionally be appended to the last column of the aforementioned constraints matrix. We will describe them and their upload formats in what follows. 
The objective matrix is written 


### Uploading and file_id's
First, import the necessary packages:

In [1]:
import numpy as np
import json
from qci_client import QciClient

In [2]:
token = "22f1f3d275d713f18f46c8020eab9848"
api_url = "https://api.qci-prod.com"
qclient = QciClient(api_token=token, url=api_url)

In [3]:
obj = np.array([[ 0. , -1.5,  0.5],
                [-1.5,  0. ,  0. ],
                [ 0.5,  0. ,  0. ]])

The constraints take the form and an explicit RHS vector can be represented as  


In [4]:
A = np.array([[1, 0, 1],
              [2, 2, 0]])
rhs = -(np.array([[1],
                [2]]))
A, rhs

(array([[1, 0, 1],
        [2, 2, 0]]),
 array([[-1],
        [-2]]))

In [5]:
constraints = np.hstack((A, rhs))
constraints

array([[ 1,  0,  1, -1],
       [ 2,  2,  0, -2]])


### API format
We encode the above in a sparse dictionary format for the upload step to the API:
### Objective 
It is essential to label the file_type value as "objective":


In [6]:
qlcbo_data = {
    'file_name': "smallest_objective.json",
    'file_config': {'objective':{"data": obj, "num_variables": 3}}
}


### Constraints
Similarly, the file_type must be labels as "constraints" for the constraints matrix.


### RHS
 Similarly, file_type should be "rhs" for the optional RHS vector.


In [7]:
qlcbo_constraints = {
    "file_name": "smallest_constraints.json",
    "file_config": {'constraints':{"data": constraints,
    "num_constraints": 2,
    "num_variables": 3}}
}


Now we can upload the various files using the client. Suppose we store the data in a variable data. Then we call upload_file to push the data to the server.


In [8]:
response_json = qclient.upload_file(qlcbo_constraints)
file_id_constraints = response_json["file_id"]

response_json = qclient.upload_file(qlcbo_data)
file_id_obj = response_json["file_id"]

In [12]:
print(file_id_constraints)

66032d8b38d25ec78cae8d54




We can extract the file_id for later use. Triggering a job to run requires the file_id to tell the backend which data to use. We cover this step in the Running tab.
## Running
Running a job involves two key steps to build parameters for the job:
1. Building a job body to submit. 
2. Providing a job_type.

### Building the job_body
The job_body is a dictionary that contains the file_id's and parameter data for running the job.  All job bodies must contain the following data fields, which can be leveraged by the user to track jobs. 



It is easiest to use `qci.build_job_body()` to construct a job_body. For example,
## *Something referring to the alpha parameter*

In [9]:
job_body = qclient.build_job_body(
    job_type="sample-constraint",
    job_params = {"sampler_type": "dirac-1", "alpha": 2, "nsamples": 5},
    constraints_file_id=file_id_constraints,
    objective_file_id=file_id_obj)

In [10]:
qclient.download_file(file_id=file_id_constraints)

{'file_id': '66032d8b38d25ec78cae8d54',
 'num_parts': 1,
 'num_bytes': 373,
 'file_name': 'smallest_constraints.json',
 'file_config': {'constraints': {'num_constraints': 2,
   'num_variables': 4,
   'data': [{'i': 0, 'j': 0, 'val': 1},
    {'i': 0, 'j': 2, 'val': 1},
    {'i': 0, 'j': 3, 'val': -1},
    {'i': 1, 'j': 0, 'val': 2},
    {'i': 1, 'j': 1, 'val': 2},
    {'i': 1, 'j': 3, 'val': -2}]}}}


This returns a job_body with the file_id fields appended to the above dictionary. Each of these file_id's was obtained after uploading the corresponding file in the Uploading section. 
Now we can trigger a job using the following command:


In [11]:
job_response = qclient.process_job(job_body=job_body, job_type="sample-constraint")

Dirac allocation balance = 0


AssertionError: Error: {"message":"File at field job_submission.problem_config.quadratic_linearly_constrained_binary_optimization.constraints_file_id does not have same number of variables as file at field job_submission.problem_config.quadratic_linearly_constrained_binary_optimization.objective_file_id"}. Received error code 400


By default, process_job polls for a job to finish. To extract results from a successful job


In [ ]:
results_list = list(job_response['results']) # defaults to generator
print(results_list[0]['energies'])  # first entries, lowest energy
print(results_list[0]['samples']).
